In [ ]:
import pandas as pd
import pathlib
import numpy as np
import urllib.request
import joblib

In [ ]:
hash = "64cfc46ecd92f1956af199c81f8ecf4dc292718f"
file_url = f"https://raw.github.com/WayScience/phenotypic_profiling_model/{hash}/2.train_model/models/log_reg_model.joblib"

In [ ]:
log_reg_model_path = pathlib.Path("log_reg_model.joblib")
urllib.request.urlretrieve(file_url, log_reg_model_path)

In [ ]:
log_reg_model = joblib.load(log_reg_model_path)

In [ ]:
plate_path = pathlib.Path("/media/roshankern/63af2010-c376-459e-a56e-576b170133b6/data/cell-health-nuc-per-plate-normalized/SQ00014613_normalized_single_cell.csv.gz")
plate_data = pd.read_csv(plate_path, compression="gzip")

In [29]:
plate_data = plate_data.sort_values(["Metadata_Reagent"])
plate_data

,Location_Center_X,Location_Center_Y,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Plate_Map_Name,Metadata_DNA,Metadata_Reagent,Metadata_Reagent_Replicate,Metadata_Model,...,efficientnet_1270,efficientnet_1271,efficientnet_1272,efficientnet_1273,efficientnet_1274,efficientnet_1275,efficientnet_1276,efficientnet_1277,efficientnet_1278,efficientnet_1279
469681,638.858824,1557.611765,SQ00014613,A3,4,SQ00014613_A3_04,../../../cell-health/SQ00014613/Images/r01c03f...,AKT1-1,1,efficientnet,...,-1.207928,1.060696,-0.365383,-1.028399,-0.292990,-0.505734,-1.771102,0.742637,-1.343861,0.893309
281875,1422.086294,466.304569,SQ00014613,O22,9,SQ00014613_O22_09,../../../cell-health/SQ00014613/Images/r15c22f...,AKT1-1,1,efficientnet,...,-0.623356,-1.100984,-0.976968,-1.120897,0.270630,-0.935060,0.864803,0.870286,-0.392306,-0.440041
281876,967.480769,449.929487,SQ00014613,O22,9,SQ00014613_O22_09,../../../cell-health/SQ00014613/Images/r15c22f...,AKT1-1,1,efficientnet,...,-0.107333,-0.874721,-0.232155,-0.348915,0.814118,-0.525089,-0.170338,0.725209,-0.854882,-0.328101
281877,1287.563636,457.387879,SQ00014613,O22,9,SQ00014613_O22_09,../../../cell-health/SQ00014613/Images/r15c22f...,AKT1-1,1,efficientnet,...,-1.231907,-0.373770,-0.607579,-0.562873,-0.583538,1.275818,0.080123,0.680419,0.388314,0.329416
281878,2065.248588,460.689266,SQ00014613,O22,9,SQ00014613_O22_09,../../../cell-health/SQ00014613/Images/r15c22f...,AKT1-1,1,efficientnet,...,1.901181,-0.882287,-0.662182,-0.188880,2.228874,0.551011,0.205427,-2.143092,-0.704278,-1.080493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54319,1684.743902,93.091463,SQ00014613,P9,9,SQ00014613_P9_09,../../../cell-health/SQ00014613/Images/r16c09f...,no reagent,1,efficientnet,...,0.593108,-1.032486,-0.739066,0.013307,-0.776281,0.244221,0.640167,-0.022095,0.469810,-1.932886
54318,1857.068571,86.582857,SQ00014613,P9,9,SQ00014613_P9_09,../../../cell-health/SQ00014613/Images/r16c09f...,no reagent,1,efficientnet,...,-0.492844,-1.234711,-0.702599,0.620788,-0.382619,-0.976422,-1.431251,-0.934352,0.378064,-1.250534
54317,1974.944162,92.715736,SQ00014613,P9,9,SQ00014613_P9_09,../../../cell-health/SQ00014613/Images/r16c09f...,no reagent,1,efficientnet,...,-0.114612,-0.527324,-0.194415,-0.124316,-1.133115,-0.760887,-1.401874,-0.098548,-0.332748,-0.557500
54328,1405.615385,162.336538,SQ00014613,P9,9,SQ00014613_P9_09,../../../cell-health/SQ00014613/Images/r16c09f...,no reagent,1,efficientnet,...,-0.480373,-0.820629,-0.580234,0.491125,2.855172,1.020296,-1.355616,0.984986,-0.327624,0.153847


{'Anaphase': 0.021535679449008874,
 'Apoptosis': 0.025231888099962702,
 'Artefact': 0.09854901189711858,
 'Binuclear': 0.15346192805594644,
 'Elongated': 0.023003092767062833,
 'Folded': 0.028890602365091944,
 'Grape': 0.030945165536819152,
 'Hole': 0.02724080941704395,
 'Interphase': 0.10769727975833449,
 'Large': 0.014377253667554529,
 'Metaphase': 0.03975616361172986,
 'MetaphaseAlignment': 0.0454043839488619,
 'Polylobed': 0.1182854857135507,
 'Prometaphase': 0.09859500161411684,
 'SmallIrregular': 0.1290160164970563,
 'UndefinedCondensed': 0.03801023760074102}

In [45]:
feature_cols = [col for col in plate_data.columns if 'efficientnet' in col]
unique_reagents = plate_data["Metadata_Reagent"].unique()
prediction_classes = log_reg_model.classes_

plate_reagent_predictions = []

count = 0
for reagent in unique_reagents:
    print(reagent)
    
    reagent_feature_data = plate_data.loc[plate_data["Metadata_Reagent"] == reagent][feature_cols].values
    reagent_predictions = log_reg_model.predict_proba(reagent_feature_data)
    # average predition probabilities across perturbations for this plate
    reagent_predictions_median = np.mean(reagent_predictions, axis=0)
    # map 16 phenotypic classes to the 16 prediction values
    reagent_predcitions_data = dict(zip(prediction_classes,reagent_predictions_median))
    
    count += 1
    if count>5:
        break
    


AKT1-1
[0.02452743 0.02281649 0.07635661 0.16233325 0.02244961 0.02165408
 0.02968678 0.03091979 0.15306871 0.01862422 0.04119366 0.05101686
 0.12292271 0.08850527 0.10401951 0.02990501]
AKT1-2
[0.02196488 0.02065239 0.07671741 0.19041612 0.01743259 0.02498909
 0.03156813 0.02355629 0.12613403 0.02022886 0.03723375 0.04116675
 0.14689926 0.06335407 0.13126796 0.02641843]
ARID1B-1
[0.02091256 0.02539282 0.0602886  0.1832917  0.02428001 0.0224457
 0.03906013 0.0320881  0.23448849 0.02383606 0.03246989 0.03967808
 0.1132714  0.05886292 0.06571806 0.02391549]
ARID1B-2
[0.02085146 0.0259479  0.07194635 0.17572393 0.02208351 0.02611864
 0.03286575 0.02635963 0.1453651  0.02412075 0.03176687 0.04138644
 0.11992265 0.07711187 0.12853127 0.02989789]
ATF4-1
[0.02183671 0.0285735  0.07051865 0.18499501 0.03122834 0.02370453
 0.04042572 0.02850399 0.17320773 0.03096242 0.02914928 0.04285372
 0.14510528 0.06278144 0.06076476 0.02538889]
ATF4-2
[0.02153568 0.02523189 0.09854901 0.15346193 0.02300309